#### 간단한 안티 크롤링 회피 방법

headers를 통해 서버에 데이터를 요청하면서 웹브라우저에 대한 정보를 같이 전송하여 마치 웹브라우저를 통해 데이터를 요청하는 것처럼 보이게 합니다

#### 요청값 규칙에 따라 URL 변화시키기
데이터를 수집하려는 페이지의 URL값에서 start번호를 자유롭게 변형시키기 위해서 변수를 추가해줍니다.
- 기존 URL: "https://search.naver.com/search.naver?where=news&query=코알라"
- 변수를 추가한 URL: "https://search.naver.com/search.naver?where=news&query=코알라&start="+str(n)   --> n은 반복시키려는 기사번호(1, 11, ...)

#### 컨테이너 수집

```python
1 <ul class="type01">
2 <li id="sp_nws1">...</li>
3 <li id="sp_nws2">...</li>
4 <li id="sp_nws5">...</li>
5 <li id="sp_nws6">...</li>
6 ...(생략)
```

1. li#sp_nws1 로 선택하는 경우
    - 첫번째 기사부분의 컨테이너를 확인하면 "sp_nws1"이라는 id를 가지고 있기 때문에 li#sp_nws1이라는 선택자로 데이터를 선택할 수 있습니다. 
    - 태그의 id는 유일한 이름이기때문에 첫번째 기사 데이터에 대해서는 정상적으로 선택이 가능하지만 모든 기사를 선택할 수 없습니다.


2. ul.type01 > li 로 선택하는 경우
    - 기사를 모두 선택할 수 없기 때문에 상위 태그에서부터 상하관계로 구분해주는 방법(자식, > )을 사용하면 ul.type01 > li 와 같이 선택자를 만들 수 있습니다.
    - 이 경우 페이지 위에서 확인할 수 있는 모든 기사를 선택할 수 있습니다.
    - 가장 적합한 선택자입니다.

#### 기사별 데이터 수집

```python
1 <a class="_sp_each_url _sp_each_title" ...(생략)>
2 오스트레일리아 폭염, 목마른 코알라
3 </a>
```

1. a._sp_each_url._sp_each_title ; 2가지 클래스를 모두 가지고있는 a 태그
    - 제목 데이터 중 일부는 2가지 클래스를 모두 가지고있고, 일부는 1가지 클래스만 가지고 있는 경우가 있기 때문에 10개 기사의 제목이 모두 선택되지 않을 수 있습니다. 
    - 모든 데이터를 가리키고 있지 못하므로 좋은 선택자가 아닙니다.
    
    
2. a._sp_each_url ;"_sp_each_url" 클래스를 가지고있는 a 태그
    - 제목 외에도 "_sp_each_url" 클래스를 가지고있는 경우가 있기 때문에 기사 개수 이상의 데이터가 선택됩니다.
    - 원하는 데이터보다 더 많은 데이터를 가리키고 있으므로 좋은 선택자가 아닙니다.
    
    
3. a._sp_each_title ;"_sp_each_title" 클래스를 가지고 있는 a 태그
    - 기사 개수만큼의 제목데이터만 선택하고 있습니다.
    - 원하는 데이터를 정확히 가리키고 있으므로 적절한 선택자입니다.

In [5]:
# 소스코드 가져오고 파싱하기
import requests
from bs4 import BeautifulSoup

# 검색어 직접 입력
keyword = input('검색어를 입력하세요: ')

# 반복1: 기사번호를 변경시키면서 데이터 수집을 반복하기
# 1 ~ 100까지 10단위로 반복(1, 11, ..., 91)
for n in range(1,100,10):
    raw = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query="+keyword+"&start="+str(n), 
                       headers={'User-Agent' : 'Morzilla/5.0'})
    html = BeautifulSoup(raw.text, "html.parser")
    articles = html.select("ul.type01>li")

    # 반복2: 기사에 대해서 반복하면 세부 정보 수집하기
    # 리스트를 사용한 반복문으로 모든 기사에 대해서 제목/언론사 출력
    for ar in articles:
        title = ar.select_one("a._sp_each_title").text
        source = ar.select_one("span._sp_each_source").text
        print(title, source)

검색어를 입력하세요: 코알라
호주 산불로 위험에 처한 코알라, 뉴욕에 나타났다 ㅍㅍㅅㅅ
호주 산불로 코알라 등 동물 113종 '긴급 도움' 절실 연합뉴스
도망치기엔 느려도 너무 느렸다, 호주산불에 코알라 멸종위기 중앙일보언론사 선정
러쉬, 코알라 비누 ‘올 더 와일드 띵스’ 출시 파이낸셜뉴스
코알라를 걱정하는 당신에게 프레시안언론사 선정
호주, 상징 코알라 성병 감염 골머리…결막염·사망 위험 조선일보
도망치기엔 너무 느려...호주 산불로 코알라 멸종 위기 YTN언론사 선정
'산불 위기종' 코알라, 뉴질랜드 도입 청원…전문가는 반대 SBS
호주 산불로 코알라 '멸종위기종' 분류될 수도 아시아경제언론사 선정
"물 주세요".. 산불로 고통받던 코알라, 도로 내려와 도움 청해 파이낸셜뉴스언론사 선정
'산불로 멸종 위기' 코알라, 뉴질랜드 도입 청원 제기 연합뉴스
최악의 호주 산불 6개월째 계속…코알라 멸종 위기 TV조선언론사 선정
불바다된 호주…코알라 멸종 위기? KBS언론사 선정
호주산불에 멸종위기 코알라, 이번엔 대규모 벌목에 학살 연합뉴스
“코알라 산 채로?” 혐오 조장하는 ‘신종 코로나 가짜뉴스’ 총정리 한겨레언론사 선정
호주 산불로 코알라 등 동물 113종 '긴급 도움' 필요 YTN사이언스
"우한폐렴, 야생동물서 비롯…'메뉴판에 코알라' 식문화 바꿔야" 연합뉴스언론사 선정
그을린 코알라, 미리 본 야생동물의 ‘기후 종말’ 한겨레언론사 선정
화마에서 구출된 코알라를 생각하며 연합뉴스
[세계NOW] "호주 산불로 코알라 멸종위기, 현재 상황은?" YTN언론사 선정
‘퀸’ 브라이언 메이, 호주 산불피해 코알라 안고 기타연주 “감동적”[해외... 마이데일리
호주 산불서 구조된 영상 속 코알라 끝내 숨져 뉴시스언론사 선정
최악 산불 아직도 '진행형'…코알라 멸종 위기 MBC
불바다 갇힌 '느릿느릿' 코알라…"멸종될 수도" MBC
호주 산불서 구조된 영상 속 코알라, 화상으로 결국 안락사 연합뉴스언론사 선정
"평범한 대학생을 데이터 사이언스 전문가로", 코알라UNIV 